In [1]:
import os
import glob
import sys
import json
import shutil
import pickle
import os.path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict
from os.path import join, exists



In [2]:
FEATURES_FOLDER = "/home/cdragancea/MastersThesis/FBACode/analyze"


In [3]:
stats = dict()
analyze_summary = dict()

with open(f"{FEATURES_FOLDER}/analyze_summary.json", "r") as fin:
    analyze_summary = json.load(fin)

counter = 0
for project_name in os.listdir(FEATURES_FOLDER):
    # print(f"Loading overall_stats for {project_name}")
    # if project_name == "analyze_summary.json":
    # if project_name == "analyze_summary.json" or project_name == "analyze_summary2.json":
    if project_name.startswith("analyze_summary") or project_name.startswith("job_distribution"):
        continue
    if not exists(join(FEATURES_FOLDER, project_name, 'overall_stats')) or project_name not in analyze_summary:
        print(f"Skipping {project_name} because it does not have overall_stats.")
        counter += 1
        # if project_name in analyze_summary:
        #     analyze_summary.pop(project_name)
        # shutil.rmtree(join(FEATURES_FOLDER, project_name), ignore_errors=True)
        continue
    if analyze_summary[project_name]["analysis status"] != "success":
        continue

    with open(join(FEATURES_FOLDER, project_name, 'overall_stats')) as f:
        content = f.read()
    try:
        overall_stats = json.loads(content)
    except json.JSONDecodeError as e:
        str_err = str(e)
        line = int(str_err.split()[3]) - 1
    
    individual_features = json.loads('\n'.join(content.split('\n')[:line]))
    overall_features = json.loads('\n'.join(content.split('\n')[line:]))
    stats[project_name] = {
        'individual_features': individual_features,
        'overall_features': overall_features
    }

project_names = list(stats.keys())

In [4]:
feature_files_map = dict()

def load_all_feature_files():
    global feature_files_map

    files = glob.glob(f"{FEATURES_FOLDER}/**/*.ast.json", recursive = True)
    for i, file in enumerate(files):
        if i % 1000 == 0:
            print(f"Loaded {i} files")

        if i == 15000:
            break
        file_project_name = file.split("/")[6]
        if file_project_name not in stats:
            print(f"{file_project_name} project is not in stats dict")
            continue
        try:
            with open(file, "r") as fin:
                data = json.load(fin)
        except Exception as e:
            print(f"Exception: {e}")
            print(f"Skipping file: {file}")
            continue
        feature_files_map[file] = data
    
    print(f"Loaded {len(feature_files_map)} feature files")
    pickle.dump(feature_files_map, open("feature_files_map.pkl", "wb"))



In [5]:
load_all_feature_files()

Loaded 0 files
Loaded 1000 files
Loaded 2000 files
Loaded 3000 files
Loaded 4000 files
Loaded 5000 files
Loaded 6000 files
Loaded 7000 files
Loaded 8000 files
Loaded 9000 files
Loaded 10000 files
Loaded 11000 files
Loaded 12000 files
Loaded 13000 files
Loaded 14000 files
Loaded 15000 files
Loaded 15000 feature files


In [6]:
loaded_feature_files_map = pickle.load(open("feature_files_map.pkl", "rb"))